<a href="https://colab.research.google.com/github/liu-bioinfo-lab/general_AI_model/blob/main/runbook_tutorial_general_genomic_AI_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please open Google Colab notebook under TPU/GPU setting : **Runtime -> Change runtime type**

In [6]:
!git clone https://github.com/liu-bioinfo-lab/general_AI_model.git
%cd general_AI_model

Cloning into 'general_AI_model'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 118 (delta 1), reused 0 (delta 0), pack-reused 111 (from 2)
Receiving objects: 100% (118/118), 68.61 MiB | 14.27 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (92/92), done.
/content/general_AI_model/general_AI_model


In [12]:
import os
import gdown
from src.model import build_model
import argparse
import torch

if "COLAB_TPU_ADDR" in os.environ and os.environ["COLAB_TPU_ADDR"]:
    import torch_xla
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.xla_multiprocessing as xmp
!pip install kipoiseq==0.5.2 --quiet > /dev/null
import kipoiseq
from kipoiseq import Interval
import pyfaidx
import pickle
import numpy as np

**Download Model**

In [8]:
os.makedirs('models', exist_ok=True)
model_path = 'models/ckpt.pt'
gdown.download('https://drive.google.com/uc?id=1aTpGvAUkvaxsDP_isA2n2Udbfqa9walW', model_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1aTpGvAUkvaxsDP_isA2n2Udbfqa9walW
From (redirected): https://drive.google.com/uc?id=1aTpGvAUkvaxsDP_isA2n2Udbfqa9walW&confirm=t&uuid=f97b15e6-4895-48d4-9e0e-3cdf99af8a8e
To: /content/general_AI_model/general_AI_model/models/ckpt.pt
100%|██████████| 468M/468M [00:06<00:00, 68.9MB/s]


'models/ckpt.pt'

In [9]:

### The following codes are copied from https://github.com/deepmind/deepmind-research/blob/master/enformer/enformer-usage.ipynb
fasta_file = '/root/data/genome.fa'
!mkdir -p /root/data
!wget -O - http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz | gunzip -c > {fasta_file}
pyfaidx.Faidx(fasta_file)
!ls /root/data
class FastaStringExtractor:
    def __init__(self, fasta_file):
        self.fasta = pyfaidx.Fasta(fasta_file)
        self._chromosome_sizes = {k: len(v) for k, v in self.fasta.items()}

    def extract(self, interval: Interval, **kwargs) -> str:
        # Truncate interval if it extends beyond the chromosome lengths.
        chromosome_length = self._chromosome_sizes[interval.chrom]
        trimmed_interval = Interval(interval.chrom,
                                    max(interval.start, 0),
                                    min(interval.end, chromosome_length),
                                    )
        # pyfaidx wants a 1-based interval
        sequence = str(self.fasta.get_seq(trimmed_interval.chrom,
                                          trimmed_interval.start + 1,
                                          trimmed_interval.stop).seq).upper()
        # Fill truncated values with N's.
        pad_upstream = 'N' * max(-interval.start, 0)
        pad_downstream = 'N' * max(interval.end - chromosome_length, 0)
        return pad_upstream + sequence + pad_downstream

    def close(self):
        return self.fasta.close()
fasta_extractor = FastaStringExtractor(fasta_file)

--2025-06-09 03:56:50--  http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 938.09M  25.9MB/s    in 41s     

2025-06-09 03:57:32 (22.8 MB/s) - written to stdout [983659424/983659424]

genome.fa  genome.fa.fai


In [10]:
# Download a downsampled GM12878 ATAC-seq for example
os.makedirs('tmp_save', exist_ok=True)
atac_path = 'tmp_save/GM12878_ATAC.pickle'
gdown.download('https://drive.google.com/uc?id=1ua-fQHYjPH658oEKEpIaDBHNFbzsO1m0', atac_path, quiet=False)
with open(atac_path, 'rb') as f:
    atac_data = pickle.load(f)

Downloading...
From (original): https://drive.google.com/uc?id=1ua-fQHYjPH658oEKEpIaDBHNFbzsO1m0
From (redirected): https://drive.google.com/uc?id=1ua-fQHYjPH658oEKEpIaDBHNFbzsO1m0&confirm=t&uuid=7b2894df-0fc9-4536-b375-3ce531b701cd
To: /content/general_AI_model/general_AI_model/tmp_save/GM12878_ATAC.pickle
100%|██████████| 192M/192M [00:01<00:00, 119MB/s]
<ipython-input-10-32c361593af3>:6: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  atac_data = pickle.load(f)


In [11]:
def one_hot_encode(sequence):
    return kipoiseq.transforms.functional.one_hot_dna(sequence).astype(np.float32)
def pad_seq_matrix(matrix, pad_left, pad_right, pad_len=300):
    # add flanking region to each sample
    dmatrix = np.concatenate((pad_left, matrix[:, :, -pad_len:]), axis=0)[:-1, :, :]
    umatrix = np.concatenate((matrix[:, :, :pad_len], pad_right), axis=0)[1:, :, :]
    return np.concatenate((dmatrix, matrix, umatrix), axis=2)

def pad_signal_matrix(matrix,pad_dnase_left,pad_dnase_right, pad_len=300):
    dmatrix = np.vstack((pad_dnase_left, matrix[:, -pad_len:]))[:-1, :]
    umatrix = np.vstack((matrix[:, :pad_len], pad_dnase_right))[1:, :]
    return np.hstack((dmatrix, matrix, umatrix))


def prepare_input(
    fasta_extractor,
    chrom, start, end,
    atac_data
):
    """
    Generate the inputs to the model
    Args:
        fasta_extractor (kipoiseq.extractors.FastaStringExtractor): kipoiseq.extractors.FastaStringExtractor object
        chrom (str), start (int), end (int): Specify a genomic region (chromosome, start genomic position, and end genomic position),
                and the size of the genomic region should be divisible by 1000.
        dnase (ndarray): a numpy array representing DNase-seq signals in the same chromosome

    Returns:
        torch.Tensor: a torch tensor (N x 5 x 1600) representing input genomic sequences and DNase-seq, where N represents the number of
                1kb sequences in the input genomic region
    """
    assert end - start == 500000

    start = start - 50000
    end = end + 50000

    if isinstance(chrom, str):
        try:
            chrom_atac = int(chrom.replace('chr',''))
        except Exception:
            chrom_atac = chrom.replace('chr','')
    else:
        chrom_atac = chrom

    if start>=end:
        raise ValueError('the start of genomic region should be small than the end.')
    if start < 300 or end+300 > atac_data[chrom_atac].shape[-1]:
        raise ValueError('please leave enough flanking region.')

    target_interval = kipoiseq.Interval(chrom, start, end)
    sequence_one_hot = one_hot_encode(fasta_extractor.extract(target_interval))
    sequence_matrix = sequence_one_hot.reshape(-1, 1000, 4).swapaxes(1, 2)

    pad_interval = kipoiseq.Interval(chrom, start - 300, start)
    seq_pad_left = np.expand_dims(one_hot_encode(fasta_extractor.extract(pad_interval)).swapaxes(0, 1), 0)
    pad_interval = kipoiseq.Interval(chrom, end, end + 300)
    seq_pad_right = np.expand_dims(one_hot_encode(fasta_extractor.extract(pad_interval)).swapaxes(0, 1), 0)
    seq_input = pad_seq_matrix(sequence_matrix, seq_pad_left, seq_pad_right)


    pad_atac_left=atac_data[chrom_atac][:,start-300:start].toarray().squeeze(0)
    pad_atac_right = atac_data[chrom_atac][:,end:end+300].toarray().squeeze(0)
    center_atac = atac_data[chrom_atac][:,start:end].toarray().squeeze(0).reshape(-1, 1000)

    atac_input = np.expand_dims(pad_signal_matrix(center_atac ,pad_atac_left,pad_atac_right), 1)

    inputs = torch.tensor(np.concatenate((seq_input, atac_input), axis=1)).float()
    return inputs.unsqueeze(0)


def extract_outputs(outputs):
    rep_1d, rep_2d, outs, extra_outs = [x.cpu().detach().numpy() for x in outputs]

    output_dict = {}
    output_dict['1D rep'] = rep_1d
    output_dict['2D rep'] = rep_2d

    modalities=['epi', 'rna', 'bru', 'microc', 'hic','intact_hic','rna_strand',
                'extra_tf', 'tt', 'groseq', 'grocap', 'proseq','netcage','starr']
    for i, d in enumerate(outs + extra_outs):
        output_dict[modalities[i]] = d

    return output_dict


In [14]:
def parser_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--bins', type=int, default=600)
    parser.add_argument('--crop', type=int, default=50)
    parser.add_argument('--embed_dim', default=960, type=int)
    args = parser.parse_args([])
    return args

def get_args():
    args = parser_args()
    return args

args = get_args()


if "COLAB_TPU_ADDR" in os.environ and os.environ["COLAB_TPU_ADDR"]:
    device = xm.xla_device()
else:
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = build_model(args)
model.load_state_dict(torch.load(model_path, map_location='cpu'),strict=True)
model.eval()
model.to(device)
device

device(type='cpu')

In [ ]:
# specify an interested region

chrom, start, end = ['chr1', 1500000, 2000000]

input_x = prepare_input(
    fasta_extractor,
    chrom, start, end,
    atac_data
).to(device)

rep_1d, rep_2d, outputs, external_outputs = model(input_x,return_rep=True)

In [ ]:
rep_1d.shape

torch.Size([1, 600, 960])